In [2]:
!pip install openai -q
# !pip uninstall openai -q
# !pip install backoff
# !pip install tenacity

In [3]:
import openai
import pandas as pd
import numpy as np
from getpass import getpass

openai.api_key = getpass()

In [4]:
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDINGS_MODEL = "text-embedding-ada-002"

In [ ]:
%cd Patents and software

In [38]:
patent_df = pd.read_csv('patent_three (4).csv')
software_df = pd.read_csv('software_catalog (2).csv')
# patent_df

In [ ]:
patent_df

In [44]:
technology_info_list = ['technology area', 'Technology area text', 'Patent name', 'Purpose', 'overview',	'Technology',	'benefits',	'Applications',	'Technology Details']
software_info_list = ['software area', 'area', 'software name', 'overview']

In [75]:
software_file_name = 'software name_embeddings.csv'
technology_file_name = 'technology_embeddings2.csv'

In [96]:
class database:

    def __init__(self, df, info_list, name, search_term_vector, no_to_get, file_name):
        self.df = df
        self.info_list = info_list
        self.col_name = name
        self.col_to_get = 'details_to_convert'
        self.search_term_vector = search_term_vector
        self.no_to_get = no_to_get
        self.file_name = file_name

    def func_get_details(self):
          for i in range(len(self.info_list)):
            if i == 0:
                self.df[self.col_to_get] = self.df[self.info_list[i]]
            else:
                self.df[self.col_to_get] += ' ' + self.df[self.info_list[i]]

    def func_get_embeddings(self):
        # patent_technology_df
        self.df['embedding'] = self.df[self.col_to_get].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
        # self.df.to_csv(self.col_name+'_embeddings.csv')

    def func_load_embeddings_from_file_name(self):
        self.df = pd.read_csv(self.file_name)

    def func_load_embeddings(self):
        self.df = pd.read_csv(self.col_name+'_embeddings.csv')

    def func_get_sim(self):
        self.df["similarities"] = self.df['embedding'].apply(lambda x: cosine_similarity(x, search_term_vector))
        self.df_sorted = self.df.sort_values("similarities", ascending=False).head(20)
        # self.df_sorted.to_csv(self.col_name+'_sorted.csv')

    def func_get_feed(self):
        self.df_feed = self.df_sorted.iloc[0:self.no_to_get][self.col_to_get].values

    def func_get_setup(self):
        self.func_get_details()
        self.func_get_embeddings()
        self.func_get_sim()
        self.func_get_feed()

    def func_load_setup(self):
        self.func_get_details()
        self.func_load_embeddings()
        self.func_get_sim()
        self.func_get_feed()

    def func_query(self):
        self.func_get_sim()
        self.func_get_feed()


In [47]:
def get_keywords(text):
    return openai.Completion.create(
        prompt='Extract keywords from the following text and output as a csv file: '+text,
        temperature=1,
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        model=COMPLETIONS_MODEL
    )["choices"][0]["text"].strip(" \n")

In [71]:
style = input('Enter a style: ')
type_of_English = input('Enter a type of English: ')
question = input('Enter a question: ')

keywords = get_keywords(question)

# semantic search
search_term_vector_keywords = get_embedding(keywords, engine="text-embedding-ada-002")
# search_term_vector
search_term_vector_question = get_embedding(question, engine="text-embedding-ada-002")
# search_term_vector

In [59]:
technology_embeddings_new = pd.read_csv('technology_embeddings2.csv')
software_embeddings_new = pd.read_csv('software name_embeddings.csv')

In [67]:
technology_embeddings_new['embedding'].values.shape
software_embeddings_new['embedding'].values[0]

'[-0.00906955637037754, 0.0009675791370682418, 0.002318764803931117, -0.02009824477136135, -0.012076757848262787, 0.017440399155020714, -0.01461815182119608, 0.0071926238015294075, -0.001700541703030467, -0.023441102355718613, 0.013371430337429047, 0.024194616824388504, -0.024838527664542198, 0.01185070350766182, -0.005024561192840338, -0.00016825602506287396, 0.01907072775065899, -0.0020584603771567345, 0.0026920964010059834, 0.0017065355787053704, 0.003952518105506897, 0.010905387811362743, -0.0008404238033108413, -0.008206441067159176, 0.011652050539851189, 0.011419146321713924, 0.02477002516388893, -0.01475515402853489, -0.017714405432343483, -0.010377928614616394, 0.008137940429151058, -0.03205855190753937, -0.0012398713734000921, -0.010782085359096527, -0.010364227928221226, -0.0007432377897202969, 0.01824871450662613, -0.020659955218434334, -0.014809954911470413, -0.0073981271125376225, 0.012275410816073418, 0.028523890301585197, 0.0006944306660443544, 0.015179861336946487, -0.0

In [ ]:
software_obj = database(software_df, software_info_list, 'software name', search_term_vector_question, no_to_get, software_file_name)
software_obj.func_load_setup()

In [97]:
# search_term_vector = 
no_to_get = 10

patent_obj = database(patent_df, technology_info_list, 'technology', search_term_vector_keywords, no_to_get, technology_file_name)
software_obj = database(software_df, software_info_list, 'software name', search_term_vector_keywords, no_to_get, software_file_name)
patent_obj.func_get_setup()
software_obj.func_get_setup()

In [121]:
patent_obj.no_to_get = 10
software_obj.no_to_get = 10
patent_obj.col_to_get = 'Technology'
software_obj.col_to_get = 'software name'
# software_obj.col_to_get 
patent_obj.func_get_feed()
software_obj.func_get_feed()

In [112]:
patent_obj.df_sorted.iloc[0:no_to_get][patent_obj.col_to_get].values

array(['The goal of COTSAT as a technology demonstration unit is to demonstrate the ability for drastic cost reduction in spacecraft design and to develop methods and technologies for maximizing reuse of developed spacecraft hardware, software and related technology on future missions. This approach will enable for rapid response capabilities given advances in rapid prototyping. \n\nCOTSAT consists of the following sub-systems: \n\n-    An artificial environment container, which comprises much of the satellite structure, is used to contain the single atmosphere  environment. The artificial atmosphere container is used to replicate an Earth-like atmosphere, allowing the use of Commercial-Off-The Shelf (COTS) hardware and electronics which were not necessarily originally designed to operate in the vacuum environment of space.\n\n-   A key design element in the bus structure of COTSAT is the modular platform upon which the bus is assembled. This structure allows for a logic-flow integrati

In [122]:
# print(patent.df_feed)

context = ''
for i in patent_obj.df_feed:
    context += ', ' + i
for i in software_obj.df_feed:
    context += ', ' + i

print(context)

, The goal of COTSAT as a technology demonstration unit is to demonstrate the ability for drastic cost reduction in spacecraft design and to develop methods and technologies for maximizing reuse of developed spacecraft hardware, software and related technology on future missions. This approach will enable for rapid response capabilities given advances in rapid prototyping. 

COTSAT consists of the following sub-systems: 

-    An artificial environment container, which comprises much of the satellite structure, is used to contain the single atmosphere  environment. The artificial atmosphere container is used to replicate an Earth-like atmosphere, allowing the use of Commercial-Off-The Shelf (COTS) hardware and electronics which were not necessarily originally designed to operate in the vacuum environment of space.

-   A key design element in the bus structure of COTSAT is the modular platform upon which the bus is assembled. This structure allows for a logic-flow integration of compon

In [ ]:
# style = input('Enter a style: ')
# type_of_English = input('Enter a type of English':)
# question = input('Enter a question: ')

In [109]:
prompt = f"""Answer the following question using only the context below. Answer in the style of {style} in {type_of_English}. 

Provide the patent number, author and location of invention with each technology used.

Each comma in the context separates each technology.

Context:
{context}

Q: {question}
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=1,
    max_tokens=2200,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'I believe that a mission to Europa for a rover that can cut ice can be successfully achieved within the proposed mission budget of $7.5 billion. We have the tools and technologies to make this mission achievable, with the use of robotics automation and control technologies such as Cost Optimized Test of Spacecraft Avionics and Technologies (COTSAT) (Reference Number: TOP2-267 and Case Number: ARC-17599-1) and aerospace technologies such as Multi-Attribute Task Battery (MATB-II)(Reference Number: LAR-17835-1 and Case Number: MSC-25505-1). Both of these technologies are patented and are at Technology Readiness Levels (TRLs) 2-5. We could make use of these technologies to minimise the mass of the rover, as well as reducing the chance of mission failure due to thermal problems.'